In [1]:
#LIBRARIES
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import warnings 
warnings.filterwarnings('ignore')
from scipy import stats  
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
from sklearn.impute import SimpleImputer
from sklearn import linear_model
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [2]:
from neuralprophet import NeuralProphet
import pickle

In [3]:
get_ipython().system('pip install db-dtypes')

In [4]:
pip install neuralprophet

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/opt/homebrew/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


In [5]:
get_ipython().system('pip install db-dtypes')

In [6]:
get_ipython().system('pip install neuralprophet')

In [7]:
#LIBRARIES
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import warnings 
warnings.filterwarnings('ignore')
from scipy import stats  
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
from sklearn.impute import SimpleImputer
from sklearn import linear_model
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from neuralprophet import NeuralProphet
import pickle

In [8]:
get_ipython().system('pip install db-dtypes')

In [9]:
get_ipython().run_cell_magic('capture', '', "%logstop\n%logstart -t -r -q ipython_command_log.py global\n\n#- IRONHACKS RESEARCH TRACKING CODE\n#----------------------------------\n# The following code is used to help our research team understand how you \n# our notebook environment. We do not collect any personal information with\n# the following code, it is used to measure when and how often you work on\n# your submission files.\n\nimport os\nfrom datetime import datetime\nimport IPython.core.history as history\n\nha = history.HistoryAccessor()\nha_tail = ha.get_tail(1)\nha_cmd = next(ha_tail)\nsession_id = str(ha_cmd[0])\ncommand_id = str(ha_cmd[1])\ntimestamp = datetime.utcnow().isoformat()\nhistory_line = ','.join([session_id, command_id, timestamp]) + '\\n'\nlogfile = open(os.environ['HOME']+'/ipython_session_log.csv', 'a')\nlogfile.write(history_line)\nlogfile.close()\n")

In [10]:
#LIBRARIES
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import warnings 
warnings.filterwarnings('ignore')
from scipy import stats  
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
from sklearn.impute import SimpleImputer
from sklearn import linear_model
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from neuralprophet import neuralprophet
import pickle

ImportError: cannot import name 'neuralprophet' from 'neuralprophet' (/opt/homebrew/lib/python3.9/site-packages/neuralprophet/__init__.py)

In [11]:
get_ipython().system('pip install db-dtypes')

In [12]:
get_ipython().system('pip install neuralprophet')

In [13]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [14]:
# Let's look at the unemployment_data table
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
"""

In [15]:
query_job = bigquery_client.query(query)
unemployment_data = query_job.to_dataframe()
unemployment_data.head(3)

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,...,gender_female,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white
0,f013068de98db1470bd986137a0c6d23,20220416,16,18003,900,"Census Tract 9, Allen County, Indiana",22,0,<NA>,14,...,<NA>,<NA>,0,0,0,11,0,0,<NA>,<NA>
1,21957d5517323845818d87623589e1ba,20220319,12,18089,10400,"Census Tract 104, Lake County, Indiana",111,0,<NA>,108,...,76,35,0,0,0,<NA>,<NA>,0,0,0
2,6a5609f385912113b6f1014b958ed748,20220326,13,18089,11500,"Census Tract 115, Lake County, Indiana",39,0,<NA>,<NA>,...,<NA>,<NA>,0,0,0,39,0,0,0,0


In [16]:
#Number of rows in unemployment_data
len(unemployment_data.index)

16833

In [17]:
# Let's look at the wage_data table
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.wage_data`
"""

In [18]:
query_job = bigquery_client.query(query)
wage_data = query_job.to_dataframe()
wage_data.head(3)

,uu_id,countyfips,tract,tract_name,average_wage
0,585f8731c2255d6b3f817a31180848b9,18177,200,"Census Tract 2, Wayne County, Indiana",6612.00
1,8c9d2aa90948679972a9382aadcc6001,18177,900,"Census Tract 9, Wayne County, Indiana",9883.25
2,0f3d45341a5b113b813ffb7be7f58bab,18183,50300,"Census Tract 503, Whitley County, Indiana",13992.25


In [19]:
#Number of rows in wage_data
len(wage_data.index)

525

In [20]:
# Let's look at the prediction_list table
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.prediction_list`
"""

In [21]:
query_job = bigquery_client.query(query)
prediction_list = query_job.to_dataframe()
prediction_list.head(3)

,uu_id,week_number
0,5bf51fc2e162d6faf9e3cf79e4198378,44
1,420b44cc7e3f55d738df565421e59941,44
2,e39c66ecceec76ee8f9f811fa4a2d246,44


In [22]:
#Number of rows in wage_data
len(prediction_list.index)

525

In [23]:
#LIBRARIES
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import warnings 
warnings.filterwarnings('ignore')
from scipy import stats  
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
from sklearn.impute import SimpleImputer
from sklearn import linear_model
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [24]:
get_ipython().system('pip install db-dtypes')

In [25]:
get_ipython().system('pip install neuralprophet')

In [26]:
from neuralprophet import NeuralProphet
import pickle

In [27]:
get_ipython().system('pip install neuralprophet')

In [28]:
from neuralprophet import NeuralProphet
import pickle

In [29]:
pip install localpip 
localpip install neuralprophet

SyntaxError: invalid syntax (1608793089.py, line 1)

In [30]:
get_ipython().system('pip install localpip')
localpip install neuralprophet

SyntaxError: invalid syntax (2720801069.py, line 2)

In [31]:
get_ipython().system('pip install localpip')
get_ipython().system('localpip install neuralprophet')

Traceback (most recent call last):
  File "/opt/anaconda3/bin/localpip", line 5, in <module>
    from localpip.__main__ import main
  File "/opt/anaconda3/lib/python3.9/site-packages/localpip/__main__.py", line 5, in <module>
    from ctypes import Structure, byref, windll
ImportError: cannot import name 'windll' from 'ctypes' (/opt/anaconda3/lib/python3.9/ctypes/__init__.py)


In [32]:
from neuralprophet import NeuralProphet
import pickle

In [33]:
from neuralprophet import NeuralProphet
import pickle